# 12. Plitke neuronske mreže

Branislav Gerazov, FEEIT, CMUS

Koristeći samo jedan neuron, model mašinskog učenja može rešavati samo linearne probleme. Drugim rečima, ako želimo da naučimo funkciju, ona može generisati samo linearnu funkciju - **linearna regresija**, a u slučaju klasifikacije neuron može razdvojiti dve klase samo linijom u 2D ili ravni u prostoru sa višim dimenzijama - **logistička regresija**.

Dakle, **nelinearni problemi** koji se obično sreću u praksi ne mogu se rešiti sa jednim neuronом.
Zbog toga se softverski neuroni gotovo uvek koriste u arhitekturama u kojima su više neurona međusobno povezani.
Ove strukture nazivamo **neuronske mreže**.

## 12.0. Arhitektura plitkih neuronskih mreža

U najjednostavnijem slučaju, neuronske mreže imaju jedan **skriveni sloj** i jedan **izlazni sloj** neurona, kao što je prikazano na slici.
Takvi modeli se nazivaju **plitke neuronske mreže**.


<img align="middle" alt="Colored_neural_network" src="https://github.com/VALENCEML/eBOOK/blob/main/EN/12/12_Colored_neural_network.png?raw=1" width="300px" style="display:block; margin-left: auto; margin-right: auto;">

**Slika 1.** Plitka neuronska mreža sa jednim skrivenim slojem koji se sastoji od četiri neurona (plavo) i jednim izlaznim slojem koji se sastoji od 2 neurona (zeleno). Ulazni podaci (karakteristike) ovde imaju 3 dimenzije i obično se prikazuju kao ulazni sloj (crveno).

* [Wikimedia - Veštačka neuronska mreža](https://commons.wikimedia.org/w/index.php?curid=24913461)


Kao što se može videti na slici 1, svaki neuron skrivenog sloja je povezan sa svakim koeficijentom ulaznog vektora $\mathbf{x}$.
Svaki neuron u izlaznom sloju je zauzvrat povezan sa svakim neuronом u skrivenom sloju.

Zbog ove međusobne povezanosti, ovaj tip slojeva se naziva **potpuno povezan** ili **gust** sloj.
Iako jednostavna, ova arhitektura omogućava neuronskim mrežama da modeluju bilo koju nelinearnu funkciju, tj. da budu **univerzalni aproksimatori**.

Izlaz plitke neuronske mreže za zadati ulazni vektor podataka $\mathbf{x}$ može se izračunati pomoću:

$$
    \mathbf{y}_h = f_h(\mathbf{a}_h) = f_h(\mathbf{W}_h \mathbf{x}^T + \mathbf{b}_h) \, , \\
    \mathbf{y} = f_o(\mathbf{a}_o) = f_o(\mathbf{W}_o \mathbf{y}_h + \mathbf{b}_o)  \\
    \mathbf{y} = f_o(\mathbf{W}_o \cdot f_h(\mathbf{W}_h \mathbf{x}^T + \mathbf{b}_h) + \mathbf{b}_o) \, ,
$$

gde $_h$ označava parametre i izlaze dobijene iz skrivenog sloja, a $_o$ označava one iz izlaznog sloja.
Ovog puta, pošto možemo imati više neurona u svakom sloju, njihove težine su raspodeljene duž redova matrica težina $\mathbf{W}$, a njihove pristrasnosti sadržane su u kolonama vektora $\mathbf{b}$.

Izlaz mreže dobija se obradom ulaznih podataka sloj po sloj, sve dok se ne dostigne izlazni sloj mreže.
Ovaj proces se naziva **forward pass** ili **feed forward**.

* Duboke neuronske mreže imaju više skrivenih slojeva, o njima raspravljamo u sledećem poglavlju.
Ako se na ulaz dovede sekvencija od $N$ uzoraka ulaznih podataka $\mathbf{x}_n$, dobijamo:

$$
    \mathbf{Y}_h = f_h(\mathbf{A}_h) = f_h(\mathbf{W}_h \mathbf{X}^T + \mathbf{b}_h) \, , \\
    \mathbf{Y} = f_o(\mathbf{A}_o) = f_o(\mathbf{W}_o \mathbf{Y}_h + \mathbf{b}_o) \\
    \mathbf{Y} = f_o(\mathbf{W}_o \cdot f_h(\mathbf{W}_h \mathbf{X}^T + \mathbf{b}_h) + \mathbf{b}_o) \, .
$$

## 12.1. Aktivacione funkcije

Da bismo koristili algoritam gradijentnog spusta (GD) za obučavanje neuronskih mreža, sve aktivacione funkcije (izlazne nelinearnosti) neurona u mreži, kao i funkcija gubitka, moraju biti diferencijabilne.
U suprotnom ne bi bilo moguće izračunati gradijent za svaki od parametara svakog neurona u mreži.

Zato se ne može koristiti diskontinualna izlazna funkcija, npr. presecanje aktivacije sa oštrim pragom:
$$
   y = f(a) = \begin{cases}
     1 & \mbox{if } a > 0.5 \\
     0 & \mbox{otherwise}
     \end{cases} \, .
$$

U tu svrhu se koriste nekoliko kontinualnih (diferencijabilnih) funkcija.
Pogledaćemo zasebno aktivacione funkcije u skrivenim i izlaznim slojevima unutar neuronske mreže.


### Aktivacione funkcije u skrivenim slojevima

Uobičajeni izbori za nelinearnosti u skrivenom sloju su:

   - **sigmoid** - sa izlazom u opsegu od 0 do 1

$$
\sigma(a) = \frac{1}{1+e^{-a}}
$$

   - **hiperbolički tangens** - sa izlazom u opsegu od -1 do 1, i
   
$$
\tanh(a) = \frac{e^{2a} - 1}{e^{2a} + 1}
$$

   - **polu-talasni ispravljač**
$$
ReLU(a) = \begin{cases}
     a & \mbox{if } a > 0 \\
     0 & \mbox{otherwise}
\end{cases}
$$

Polu-talasni ispravljač ima nekoliko prednosti, uključujući: jednostavnije izračunavanje izlaza neurona, bolje širenje gradijenta tokom procesa obuke, kao i retkost aktivacije neurona - sa slučajnim inicijalizacijama težina, polovina neurona će imati izlaz 0.
Zbog toga se često primenjuje u skrivenim slojevima neuronskih mreža.

Još jedan motiv za korišćenje ove nelinearnosti je asimetrija u odnosu na $y$-osu, što je analogno načinu rada bioloških neurona.

Prikažimo ove aktivacione funkcije u Pythonu.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

a = np.linspace(-3, 3, 100)
y_sigmoid = 1 / (1 + np.exp(-a))
y_tanh = (np.exp(2 * a) - 1) / (np.exp(2 * a) + 1)
y_relu = a.copy() 
y_relu[a < 0] = 0 
plt.plot(a, y_sigmoid, lw=2, alpha=0.7)
plt.plot(a, y_tanh, lw=2, alpha=0.7)
plt.plot(a, y_relu, lw=2, alpha=0.7)
plt.axis([-3.1, 3.1, -2.1, 2.1])
plt.grid(True)
plt.legend(["sigmoid", "tanh", "ReLU"])
plt.xlabel("Activation")
plt.ylabel("Neuron Output")

### Aktivacione funkcije u izlaznom sloju

Za neurone u izlaznom sloju, često se koriste:

   - **sigmoid** - za klasifikaciju,
   - **softmax** - za klasifikaciju sa više izlaznih klasa $J$:

$$
f(a_j) = \frac{e^{a_j}}{\sum_{j=0}^{J-1} e^{a_j}} \, ,
$$

gde je $a_j$ aktivacija neurona koji odgovara klasi $j$; softmax funkcija normalizuje zbir izlaza svih izlaznih neurona, tako da se može smatrati aproksimacijom verovatnoće svake od klasa $f(a_j) \approx P(y = j \mid \mathbf{a})$,

   - **linearna** - u regresionim modelima

$$
f(a) = a
$$

## 12.2. Funkcija gubitka

Izbor nelinearnosti u izlaznom sloju odrediće izbor funkcije gubitka mreže.
Neke funkcije gubitka imaju povoljnije karakteristike za neke izlazne nelinearnosti u odnosu na druge.

Često se koriste sledeće funkcije grešaka:

   - **srednje kvadratna greška** - osnovna funkcija greške za regresiju i binarnu klasifikaciju

$$
MSE = \frac{1}{N} \sum_{n=0}^{N-1} (y - \tilde{y})^2
$$

   - **unakrsna entropija** - ima derivat sa boljim karakteristikama u klasifikacionim modelima u kojima je izlazna nelinearnost sigmoid,

$$
CE = - \frac{1}{N} \sum_{n=0}^{N-1} y \ln \tilde{y} + (1-y) \ln(1-\tilde{y})
$$
   - **log-likelihood** - u modelima sa softmax izlaznom funkcijom.

$$
LL = - \ln \tilde{y}
$$

## 12.3. Obuka plitkih neuronskih mreža

Kao što smo rekli u prethodnom poglavlju, neuronske mreže se obično obučavaju pomoću **algoritma gradijentnog spusta (GD ili Gradiend descent)**.

Kao što je spomenuto u poglavlju 11., u plitkim i dubokim neuronskim mrežama, proračun gradijenta za podešavanje parametara $\theta$ počinje proračunom gradijenata za izlazni sloj, a zatim se vraća kroz (sve) skrivene slojeve.
Ovom procesu se daje ime **povratno propagiranje** (backpropagation).

* Proces povratnog propagiranja gradijenta je izraženiji u dubokim neuronskim mrežama koje imaju više skrivenih slojeva.

**Lančano pravilo** iz matematičke analize se koristi za izračunavanje parcijalnih izvoda u odnosu na svaki parametar mreže:
$$
   \frac{\partial \mathcal{L}}{\partial \theta_l} =
   \frac{\partial \mathcal{L}}{\partial \tilde{y}}
   \cdot \frac{\partial \tilde{y}}{\partial y_{L-2}}
   \, \cdots \, \frac{\partial y_{l+1}}{\partial \theta_{l+1}}
   \cdot \frac{\partial y_{l}}{\partial \theta_{l}}
   \, ,
$$

gde je $y_l$ izlaz $l$-tog sloja mreže i $L$ je ukupan broj slojeva.

### Inicijalizacija parametara mreže

Obično se početne vrednosti parametara mreže određuju nasumično tokom njene inicijalizacije.
Trebali bismo shvatiti da će tačna vrednost parametara uticati na proces učenja.
Tako, ako pretpostavimo da funkcija gubitka $\mathcal{L}$ zavisi od parametra $\theta$ kako je prikazano na slici 2, onda možemo videti da bi obuka mreže bila brža ako je početna vrednost $\theta$ bliža minimumu $\mathcal{L}$, na primer u poziciji $\theta_B$ ili $\theta_C$ umesto $\theta_A$.
Naravno, ne znamo gde se nalazi minimum $\mathcal{L}$, ali može biti korisno obaviti nekoliko obuka na istoj mreži polazeći od različitih početnih vrednosti parametara.

<img align="middle" alt="Gradient descend start positions" src="https://github.com/VALENCEML/eBOOK/blob/main/EN/12/12_gradient_descend_start.png?raw=1" width="600px" style="display:block; margin-left: auto; margin-right: auto;">

Slika 2. Izbor početne vrednosti parametra $\theta$.

Problem izbora početne tačke je još veći ako funkcija gubitka $\mathcal{L}$ ima složeniju zavisnost od parametra $\theta$, kao na slici 3. Tada izbor početne vrednosti $\theta$ može biti ključan za optimalno obučavanje mreže. Iz tri ponuđene početne vrednosti, samo iz $\theta_C$ možemo doći do optimalne vrednosti $\theta$ za koju $\mathcal{L}$ ima najmanju vrednost, nazvanu globalni minimum, kroz gradijentni spust. U suprotnom, ako počnemo od $\theta_B$ ili $\theta_A$, proces obuke završiće se sa vrednostima $\theta$ za koje vrednost $\mathcal{L}$ ima **lokalne minimume**.

* Imajte na umu da je $\theta_B$ bliže globalnom minimumu od $\theta_C$!

<img align="middle" alt="Gradient descend global and local minimum" src="https://github.com/VALENCEML/eBOOK/blob/main/EN/12/12_gradient_descend_start_global_local.png?raw=1" width="600px" style="display:block; margin-left: auto; margin-right: auto;">

**Slika 3.** Primer početnih vrednosti parametra $\theta$ sa složenijom funkcijom gubitka.

### Stopa učenja

Kada koristimo GD, stopa učenja je jedan od najvažnijih parametara u obuci neuronskih mreža. Ako je previše velika, optimizacija može promašiti minimum funkcije greške, dok ako je previše mala, algoritmu će biti potrebno mnogo iteracija da završi obuku.

<img align="middle" alt="Gradient descend large learning rate" src="https://github.com/VALENCEML/eBOOK/blob/main/EN/12/12_gradient_descend_large_rate.png?raw=1" width="600px" style="display:block; margin-left: auto; margin-right: auto;">

<img align="middle" alt="Gradient descend small learning rate"  src="https://github.com/VALENCEML/eBOOK/blob/main/EN/12/12_gradient_descend_small_rate.png?raw=1" width="600px" style="display:block; margin-left: auto; margin-right: auto;">

Slika 4. Ilustracija uticaja koraka učenja na proces obuke: kada je korak velik, mreža će brzo ići prema minimumu funkcije gubitka, ali neće moći da ga dostigne; kada je korak mali, mreža može dostići minimum, ali obuka će zahtevati veći broj iteracija.

Kako bi se optimizovao proces učenja, obično se koristi **planiranje stope učenja**.

Jedna jednostavna strategija je da se koristi veća stopa učenja na početku obuke, kada mreža "ne zna" ništa o problemu, kako bi se postigao brži napredak.
Zatim se ovaj prvobitno veliki korak postepeno smanjuje kako bi obuka mogla da konvergira bliže minimumu funkcije gubitka.

Postoje i napredniji algoritmi, kao što je **Adam**, koji se često koristi u obuci neuronskih mreža. On uzima u obzir prve i druge momente, tj. brzinu i ubrzanje promene gradijenta u odnosu na prethodne iteracije za prilagođavanje koraka učenja.

Jedan od glavnih problema u obuci neuronskih mreža je mogućnost da se proces zaglavi u lokalnom minimumu, uz trošak propuštanja globalnog minimuma.
Ovo je izraženije u manjim neuronskim mrežama.
Ovaj problem može biti rešen reinicijalizacijom koraka učenja na početnu veliku vrednost nakon određenog broja iteracija.

### Podela skupa za obuku na serije

Kod obuke neuronskih mreža, skupovi podataka su obično veliki i ne mogu se čuvati u radnoj memoriji, pa je potrebno podeliti podatke na delove tokom obuke.

Jedan ekstrem je optimizacija parametara mreže sa gradijentom izračunatim za svaki od uzoraka iz skupa za obuku pojedinačno.
Ova varijanta GD algoritma se naziva stohastički gradijentni spust.
Prolazak kroz čitav skup za obuku naziva se epoha.
Randomizacija ulaznih uzoraka se vrši nasumičnim mešanjem skupa podataka pre svake epohe, a zatim sekvencijalnim uzorkovanjem.

Međutim, izračunavanje gradijenta po uzorku ne pruža dobru procenu stvarnog gradijenta funkcije gubitka.
Rešenje je upotreba podskupa ili serije (batch) uzoraka iz skupa za obuku uzetih nasumično, kako bi se izračunao gradijent i ažurirali parametri.
Ova verzija GD algoritma se naziva **gradijentni spust serije (BGD)* i gradijentni spust mini-serije (MBGD)**.

* Tehnički postoji razlika između MBGD-a i BGD-a - kod MBGD-a se parametri ažuriraju za svaku seriju, dok se kod BGD-a to radi nakon prolaska kroz ceo skup za obuku.

Skoro uvek za obuku neuronskih mreža i drugih algoritama mašinskog učenja koristi se MBGD algoritam, ali pod imenom SGD.

## 12.4. Regularizacija u neuronskim mrežama

Kod obučavanja neuronskih mreža postoji opasnost od **preprilagođavanja** parametara neuronske mreže. To se događa kada mreža postane preterano prilagođena trening skupu, i kao rezultat toga, njene performanse na test skupu se pogoršavaju. Stoga je potrebno kažnjavati preprilagođavanje tokom procesa obučavanja. Ovo se naziva **regularizacija**.

S obzirom da se kod preprilagođavanja parametri mreže dobijaju izuzetno visoke vrednosti, jedan način za regularizaciju je da se uvede član u funkciji gubitka koji će ih kažnjavati. To se postiže uvođenjem $L^2$ norme $\theta$ modela parametara. Na primer, koristeći srednju kvadratnu grešku kao primer, imali bismo:

$$
   L(y, \tilde{y}, \theta) = L(y, \tilde{y}) + \lambda \sum_{l=0}^{L-1} (\mathbf{W}_l^T \mathbf{W}_l + \mathbf{b}_l^T \mathbf{b}_l) \, ,
$$

gde je $\lambda$ **koeficijent regularizacije**, a $\mathbf{W}_l^T \mathbf{W}_l$ i $\mathbf{b}_l^T \mathbf{b}_l$ daju zbir svih kvadriranih parametara neurona sloja $l$.

## Regresija sa plitkom neuronskom mrežom

Napravimo neuronsku mrežu kako bismo predvideli izlaz sinusne funkcije za ulazne podatke koji nisu deo trening skupa. Da bismo radili sa neuronskim mrežama, koristićemo paket `scikit-learn`.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import neural_network

Prvo, napravimo trening skup. Uzećemo 10 jednako razmaknutih vrednosti u intervalu od 0 do 2$\pi$ i izračunati sinusne vrednosti za njih. Pored sinusne funkcije, dodajemo i beli šum.

In [ ]:
xs = np.linspace(0, 2 * np.pi, 10)
np.random.seed(42)
ys = np.sin(xs) + np.random.normal(size=xs.size) * 0.2

Prikažimo dobijeni trening skup i sinusnu funkciju koja će predstavljati ciljnu funkciju koju želimo naučiti.

In [ ]:
x_axis = np.linspace(-.1, 2 * np.pi + .1, 100)
y_sin = np.sin(x_axis)

xs = np.expand_dims(xs, 1)
x_axis = np.expand_dims(x_axis, 1)

plt.figure()
plt.scatter(xs, ys)
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.grid()

Inicijalizujmo neuronsku mrežu klasom `MLPRegressor`, koja će kreirati neuronsku mrežu sa linearnom izlaznom funkcijom u neuronu izlaznog sloja.

In [ ]:
reg = neural_network.MLPRegressor(
    hidden_layer_sizes=(5),
    activation="tanh",
    solver="adam",
    alpha=0.0001,
    batch_size="auto",
    learning_rate_init=0.01,
    max_iter=50,
    tol=1e-4,
    early_stopping=False,
    random_state=42,
    verbose=True,    
    )

Sa ulaznim parametrima definišemo sledeće vrednosti:
- `hidden_layer_sizes` - broj neurona u skrivenom sloju mreže,
- `activation` -  izbor izlazne nelinearnosti neurona u skrivenom sloju mreže,
- `solver` -  izbor algoritma za učenje,
- `alpha` - koeficijent $L^2$ regularizacije, koji smo označili gore sa $\lambda$,
- `batch_size` -  veličina serije (batch-a),
- `learning_rate_init` - početna vrednost koraka učenja,
- `max_iter` - maksimalni broj epoha obučavanja mreže,
- `tol` - ako se gubitak promeni za manje od ovog praga tolerancije tokom 10 epoha, obučavanje će biti zaustavljeno uprkos tome što maksimalni broj epoha nije dostignut,
- `early_stopping` -  parametar koji omogućava rano zaustavljanje obučavanja ako gubici izračunati na validacionom podskupu počnu rasti, koji se automatski odvaja od trening skupa nasumično,
- `random_state` -  podešavanje generatora slučajnih brojeva kako bismo dobili iste parametre mreže tokom njene slučajne inicijalizacije,
- `verbose` -  nivo detalja izlaza u procesu obučavanja mreže.

Svi ovi argumenti koji opisuju arhitekturu mreže i koji kontrolišu proces obučavanja poznati su kao hiperparametri, kako bi ih razlikovali od stvarnih parametara neurona u mreži - pri čemu se ovi poslednji ažuriraju u procesu obučavanja, a ovi prvi ne.

Da bismo videli sve dostupne postavke za neuronsku mrežu, kao i više detalja o svakoj od njih, možemo otkucati `neural_network.MLPRegressor?`

Nakon što smo inicijalizovali mrežu, možemo je obučiti sledećom komandom:

In [ ]:
reg.fit(xs, ys)

Iz ispisa možemo videti da obučavanje nije konvergiralo, odnosno, gubitak je i dalje opadao pre nego što je obučavanje zaustavljeno zbog dostizanja maksimalnog broja epoha. Drugim rečima, naša mreža se **nedovoljno prilagođava (under fitting)**.

Sada možemo koristiti obučenu mrežu za predviđanje vrednosti naučene funkcije:

In [ ]:
y_pred_under = reg.predict(x_axis)

I prikazati rezultate:

In [ ]:
# %% plot results
plt.figure()
plt.scatter(xs, ys, c='k')
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.plot(x_axis, y_pred_under)
plt.grid()
plt.tight_layout()

Vidimo da je mreža relativno dobro opisala tačke trening seta, ali nije naučila ciljnu funkciju.

Da bismo ponovno obučili mrežu, inicijalizovaćemo je s novim parametrima i ponoviti obučavanje.

In [ ]:
reg = neural_network.MLPRegressor(
    hidden_layer_sizes=(5),
    activation="tanh",
    alpha=0.0001,
    learning_rate_init=0.01,
    max_iter=500,
    tol=1e-4,
    random_state=42,
    )
reg.fit(xs, ys)

Vidimo da je sada obučavanje konvergiralo. Prikazaćemo rezultate.

In [ ]:
y_pred = reg.predict(x_axis)

plt.figure()
plt.scatter(xs, ys, c='k')
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.plot(x_axis, y_pred_under, label="under fitting")
plt.plot(x_axis, y_pred, label="converged")
plt.grid()
plt.legend()

Vidimo da je mreža nakon konvergiranja došla bliže ciljnoj funkciji.

Prikažimo parametre obučene mreže:

- poslednju vrednost gubitka,
- broj epoha u obučavanju,
- težine neurona u mreži, i
- pristrasnosti neurona u mreži.

In [ ]:
print("loss", reg.loss_)
print("epochs", reg.n_iter_)
print("weights", reg.coefs_)
print("biases", reg.intercepts_)

Vidimo da je obučavanje završeno u 274 epohe, a konačna vrednost gubitka iznosi 0,059. To je manje od poslednjeg gubitka kada smo zaustavili obučavanje u epohi 50 (0,14).

Prikazaćemo i promenu gubitka tokom obučavanja.

In [ ]:
plt.plot(reg.loss_curve_)
plt.grid()

Već smo videli kako je promena jednog od hiperparametara (maksimalni broj epoha) uticala na proces obučavanja.

Uradićemo neke eksperimente kako bismo videli kako ostali hiperparametri utiču na performanse dobijene mreže.

### Eksperiment 1. Menjanje broja neurona

In [ ]:
plt.figure()
plt.scatter(xs, ys)
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.grid()
for neurons in [1, 2, 5]:
    reg = neural_network.MLPRegressor(
        hidden_layer_sizes=neurons,
        activation="tanh",
        alpha=0.0001,
        learning_rate_init=0.01,
        max_iter=500,
        tol=1e-4,
        random_state=42,
        )
    reg.fit(xs, ys)
    y_preds = reg.predict(x_axis)
    plt.plot(x_axis, y_preds, lw=2, alpha=0.5, label=neurons)

plt.legend()

Vidimo da s 1 neuronom u skrivenom sloju na izlazu mreže dobijamo njen izlazni nelinearni funkciju `tanh`. Već sa 2 neurona mreža može naučiti funkciju blisku ciljnoj. Ovo ilustruje potrebu za snagom modela da bude adekvatna problemu koji želimo da rešimo.

### Eksperiment 2. Menjanje stope učenja

In [ ]:
# %% try different learning rates
plt.figure()
plt.scatter(xs, ys)
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.grid()
for learn in [0.001, 0.01, 0.1, 1]:
    reg = neural_network.MLPRegressor(
        hidden_layer_sizes=(5),
        activation="tanh",
        alpha=0.0001,
        learning_rate_init=learn,
        max_iter=500,
        tol=1e-4,
        random_state=42,
        )
    reg.fit(xs, ys)
    y_preds = reg.predict(x_axis)
    plt.plot(x_axis, y_preds, lw=2, alpha=0.5, label=learn)

plt.legend()

Grafikon jasno pokazuje uticaj stope učenja na proces obučavanja. Za vrlo velike vrednosti (1) možemo videti da je obučavanje potpuno propalo. Bolje rezultate dobijamo za vrednosti 0,1 i 0,01. Za male vrednosti (0,001) možemo videti da obučavanje nije konvergiralo unutar istog broja epoha kao za veće vrednosti.

### Eksperiment 3. Menjanje inicijalizacije obučavanja

In [ ]:
# %% try different starting points
plt.figure()
plt.scatter(xs, ys)
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.grid()
for seed in [1, 10, 42, 100]:
    reg = neural_network.MLPRegressor(
        hidden_layer_sizes=(5),
        activation="tanh",
        alpha=0.0001,
        learning_rate_init=0.01,
        max_iter=500,
        tol=1e-4,
        random_state=seed,
        )
    reg.fit(xs, ys)
    y_preds = reg.predict(x_axis)
    plt.plot(x_axis, y_preds, lw=2, alpha=0.5, label=seed)

plt.legend()

Vidimo da zaista početna tačka utiče na konačni rezultat procesa obučavanja.

### Eksperiment 4. Regularizacija

Da bismo ilustrovali potrebu za regularizacijom mreže, hajde da vidimo šta se događa ako smanjimo toleranciju zaustavljanja i povećamo maksimalni broj epoha.

In [ ]:
reg = neural_network.MLPRegressor(
    hidden_layer_sizes=(5),
    activation="tanh",
    alpha=0.0001,
    learning_rate_init=0.01,
    max_iter=5000,
    tol=1e-9,
    random_state=42,
    )
reg.fit(xs, ys)

In [ ]:
y_pred_over = reg.predict(x_axis)

plt.figure()
plt.scatter(xs, ys, c='k')
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.plot(x_axis, y_pred_under, label="underfitting")
plt.plot(x_axis, y_pred, label="converged")
plt.plot(x_axis, y_pred_over, label="overfitting")
plt.grid()
plt.legend()

Vidimo da je mreža savršeno naučila trening set. To se zove preprilagođavanje (overfitting). To je naravno nepoželjan rezultat, jer želimo da mreža može da radi dobro na podacima koji nisu deo trening seta.

Možemo zaključiti da je veća vrednost tol zaustavila obučavanje pre nego što su se parametri mreže uskladili. Zapravo, izazvalo je rano zaustavljanje (early stopping), što je još jedna strategija za regularizaciju.

Hajde da vidimo kakav će uticaj regularizacija imati na proces obučavanja.

In [ ]:
# %% tweak regularisation
plt.figure()
plt.scatter(xs, ys)
plt.plot(x_axis, y_sin, ':', lw=4, alpha=0.2)
plt.grid()
for alpha in [1, 0.5, 0.1, 0.01]:
    reg = neural_network.MLPRegressor(
        hidden_layer_sizes=(5),
        activation="tanh",
        alpha=alpha,
        learning_rate_init=0.01,
        max_iter=10000,
        tol=1e-9,
        random_state=42,
        )
    reg.fit(xs, ys)
    y_preds = reg.predict(x_axis)
    plt.plot(x_axis, y_preds, lw=2, alpha=0.5, label=alpha)

plt.legend()

Vidimo efekat preterane regularizacije. U slučaju kada je $\lambda$ 1, mreža uopšte ne uči iz trening seta. Za prevelike vrednosti (0,01) mreža se ponovo preprilagođava. Za vrednost 0,1 dobijamo najbolje rezultate ne samo u ovom eksperimentu, već i ukupno - za sve dosadašnje eksperimente.